In [1]:
!test -f test6.grib || wget https://github.com/ecmwf/emohawk/raw/main/docs/examples/test6.grib

### GRIB handling basics

We load a GRIB file containing 6 messages from disk:

In [2]:
import emohawk

fs = emohawk.load_from("file", "test6.grib")

No GRIB data is actually loaded at this point.

#### Iteration

A GRIB data object consists of GribFields. When used in iteration these are automatically created and released when going out of scope. As a result, during the iteration only one GRIB message at a time is kept in memory:

In [3]:
for f in fs:
    print(f)

GribField(t,1000,20180801,1200,0,0)
GribField(u,1000,20180801,1200,0,0)
GribField(v,1000,20180801,1200,0,0)
GribField(t,850,20180801,1200,0,0)
GribField(u,850,20180801,1200,0,0)
GribField(v,850,20180801,1200,0,0)


#### Inspecting the contents

In [4]:
len(fs)

6

In [5]:
fs.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
2,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
3,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
4,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
5,ecmf,v,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [6]:
fs.describe()

,,level,date,time,step,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,
t,isobaricInhPa,"1000,850",20180801,1200,0,130,od,oper,an,0001
u,isobaricInhPa,"1000,850",20180801,1200,0,131,od,oper,an,0001
v,isobaricInhPa,"1000,850",20180801,1200,0,132,od,oper,an,0001


#### Slicing

Standard Python slicing is available. It does not involve any loading/copying of GRIB data. 

In [7]:
g = fs[1]
g

GribField(u,1000,20180801,1200,0,0)

In [8]:
g = fs[1:3]
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,v,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll


In [9]:
g = fs[-1]
g

GribField(v,850,20180801,1200,0,0)

#### Getting data values

In [10]:
v = fs[0].to_numpy()
v.shape

(84,)

In [11]:
v[0:4]

array([272.56417847, 272.56417847, 272.56417847, 272.56417847])

In [12]:
v = fs.to_numpy()
v.shape

(6, 84)

In [13]:
v[0:2,0:4]

array([[272.56417847, 272.56417847, 272.56417847, 272.56417847],
       [ -6.28688049,  -6.28688049,  -6.28688049,  -6.28688049]])

#### Metadata

Metadata access works both on individual fields and slices:

In [14]:
fs[0].metadata("typeOfLevel")

'isobaricInhPa'

In [15]:
fs[0:2].metadata(["level", "paramId"])

[[1000, 130], [1000, 131]]

and on all the fields:

In [16]:
fs[:4].metadata("level")

[1000, 1000, 1000, 850]

In [17]:
fs[:4].metadata(["level", "paramId"])

[[1000, 130], [1000, 131], [1000, 132], [850, 130]]

Key qualifiers can be used to prescribe the required metadata type:

In [18]:
fs[0].metadata(["centre", "centre:l", "centre:int", "centre:s", "centre:str"])

['ecmf', 98, 98, 'ecmf', 'ecmf']

Metadata access with [] only works on individual fields:

In [19]:
fs[0]["centre"]

'ecmf'

#### Selection

Selection by metadata is always creating a "view", no copying of GRIB data is involved.

In [20]:
g = fs.sel(shortName=["u", "v"], level=850)
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
1,ecmf,v,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [21]:
g = fs.sel(param="t")
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


A slice can be used to define an interval:

In [22]:
g = fs.sel(param="t", level=slice(800,1000))
g.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll
1,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


#### Xarray

Xarray conversion does not involve disk writing. Under the hood it uses cfgrib.

In [23]:
ds = fs.to_xarray()
ds

<xarray.Dataset>
Dimensions:        (number: 1, time: 1, step: 1, isobaricInhPa: 2, latitude: 7,
                    longitude: 12)
Coordinates:
  * number         (number) int64 0
  * time           (time) datetime64[ns] 2018-08-01T12:00:00
  * step           (step) timedelta64[ns] 00:00:00
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 850.0
  * latitude       (latitude) float64 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude      (longitude) float64 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
    valid_time     (time, step) datetime64[ns] ...
Data variables:
    t              (number, time, step, isobaricInhPa, latitude, longitude) float32 ...
    u              (number, time, step, isobaricInhPa, latitude, longitude) float32 ...
    v              (number, time, step, isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-02-13T11:43 GRIB to CDM+CF via cfgrib-0.9.1...

We can pass all the kwargs arguments cfgrib accepts. On top of that emohawk provides the **ignore_keys** option to omit keys from the default set of keys used by cfgrib:

In [24]:
ds = fs.to_xarray(xarray_open_dataset_kwargs={"backend_kwargs": {"ignore_keys": ["number"]}})
ds

<xarray.Dataset>
Dimensions:        (time: 1, step: 1, isobaricInhPa: 2, latitude: 7,
                    longitude: 12)
Coordinates:
  * time           (time) datetime64[ns] 2018-08-01T12:00:00
  * step           (step) timedelta64[ns] 00:00:00
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 850.0
  * latitude       (latitude) float64 90.0 60.0 30.0 0.0 -30.0 -60.0 -90.0
  * longitude      (longitude) float64 0.0 30.0 60.0 90.0 ... 270.0 300.0 330.0
    valid_time     (time, step) datetime64[ns] ...
Data variables:
    t              (time, step, isobaricInhPa, latitude, longitude) float32 ...
    u              (time, step, isobaricInhPa, latitude, longitude) float32 ...
    v              (time, step, isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-02-13T11:43 GRIB to CDM+CF via cfgrib-0.9.1...